In [1]:
import pandas as pd
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import xgboost

In [2]:
df = pd.read_csv("/Users/alperozarslan/Desktop/Course+Materials/31 - Exercise Loan Approval/data/loan_data.csv")

In [3]:
df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,male,Associate,47971.0,6,RENT,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
44996,37.0,female,Associate,65800.0,17,RENT,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
44997,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
44998,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [5]:
df.person_education.value_counts()
df.drop("previous_loan_defaults_on_file",axis=1,inplace=True)
dfdummienames = df.select_dtypes("object").columns
dummieler = pd.get_dummies(df.select_dtypes("object"),dtype=int)

In [6]:
dummieler

,person_gender_female,person_gender_male,person_education_Associate,person_education_Bachelor,person_education_Doctorate,person_education_High School,person_education_Master,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
2,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
3,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0
4,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
44996,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
44997,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0
44998,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0


In [7]:
dummytrap = []
for i in dfdummienames:
    if df[i].value_counts().count() ==2:
        dummytrap.append(i)

In [8]:
traplidummies = pd.get_dummies(df[dummytrap],drop_first=True,dtype=int)

In [9]:
normaldummie = dfdummienames.drop(dummytrap)
normaldummie= df[normaldummie]
normaldummie = pd.get_dummies(normaldummie,dtype=int)

In [10]:
sondf = pd.concat([normaldummie,traplidummies],axis=1)

In [11]:
sondf= pd.concat([sondf,df.drop(dfdummienames,axis=1)],axis=1)
kolonlarimiz = sondf.columns

In [12]:
scaler = MinMaxScaler()
x = torch.tensor(scaler.fit_transform(sondf.drop(["loan_status"],axis=1).to_numpy()),dtype=torch.float32)
y = torch.tensor((sondf["loan_status"]).to_numpy(),dtype=torch.float32).reshape(-1, 1)

In [13]:
model = nn.Sequential(
    nn.Linear(x.shape[1], 100),
    nn.ReLU(),
    nn.Linear(100, 1),
    nn.Sigmoid()
)

In [14]:
dataset = TensorDataset(x,y)
dataloader = DataLoader(dataset, batch_size=500,shuffle=True)

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lossfunc = torch.nn.BCELoss()

In [16]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)
lossfunc = lossfunc.to(device)
model.train()


Sequential(
  (0): Linear(in_features=24, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=1, bias=True)
  (3): Sigmoid()
)

In [17]:
for epoch in range(100):
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)

        predict = model(x)
        loss = lossfunc(predict, y)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        print(loss.item())

0.6668223738670349
0.6604727506637573
0.6532449126243591
0.6515782475471497
0.6462448835372925
0.638503909111023
0.6336845755577087
0.6181446313858032
0.6255616545677185
0.6162417531013489
0.6002037525177002
0.5936316847801208
0.5951493382453918
0.5957006812095642
0.5801961421966553
0.5870726108551025
0.5869055986404419
0.5893978476524353
0.5684810876846313
0.5675737261772156
0.5800067782402039
0.5698766112327576
0.5380366444587708
0.5847706198692322
0.5666311979293823
0.5517416596412659
0.5694910883903503
0.5495175719261169
0.5333831906318665
0.5696160197257996
0.5248615741729736
0.5255680680274963
0.5237593054771423
0.518470823764801
0.5304366946220398
0.5509764552116394
0.5461911559104919
0.5091032385826111
0.5145689249038696
0.503868818283081
0.49239182472229004
0.5341467261314392
0.49277907609939575
0.5232964158058167
0.5137227177619934
0.49505555629730225
0.5268323421478271
0.5337882041931152
0.48131677508354187
0.5166860222816467
0.5440337061882019
0.5239859819412231
0.517124116

In [18]:
model.eval()
with torch.no_grad():
    y_pred = model(x.to(device))
    y_pred = y_pred.cpu()
binary = np.where(y_pred > 0.5, 1, 0)

In [19]:
mean_squared_error(y.cpu(), binary)

0.084

In [20]:
# İlk Linear katmanı al
linear1 = model[0]  # çünkü Sequential içinde ilk katman model[0]

# Ağırlıkları al ve önem hesapla
with torch.no_grad():
    weights = linear1.weight  # shape: [100, input_dim]
    importance = weights.abs().mean(dim=0).cpu().numpy()
# Sıralayıp yazdır
for idx in np.argsort(importance)[::-1]:
    print(f"Feature {kolonlarimiz[idx]}: importance = {importance[idx]:.4f}")

Feature loan_percent_income: importance = 0.6651
Feature person_income: importance = 0.5239
Feature person_home_ownership_OWN: importance = 0.3533
Feature loan_int_rate: importance = 0.3400
Feature loan_amnt: importance = 0.3306
Feature person_home_ownership_MORTGAGE: importance = 0.2040
Feature person_age: importance = 0.1906
Feature person_emp_exp: importance = 0.1894
Feature person_home_ownership_RENT: importance = 0.1772
Feature loan_intent_HOMEIMPROVEMENT: importance = 0.1759
Feature person_home_ownership_OTHER: importance = 0.1728
Feature loan_intent_MEDICAL: importance = 0.1677
Feature loan_intent_EDUCATION: importance = 0.1630
Feature loan_intent_PERSONAL: importance = 0.1556
Feature person_education_Doctorate: importance = 0.1551
Feature loan_intent_DEBTCONSOLIDATION: importance = 0.1500
Feature cb_person_cred_hist_length: importance = 0.1412
Feature person_education_Master: importance = 0.1411
Feature loan_intent_VENTURE: importance = 0.1337
Feature person_education_High Scho

🎯 Genel Yorum ve Çıkarımlar

✅ 1. Gelirle İlgili Özellikler En Kritik Etmenlerdir
	•	loan_percent_income (0.6651) ve person_income (0.5239) en yüksek öneme sahip.
➤ Bu, başvurucunun gelir durumu ve bu gelirin kredi tutarına oranının modelde belirleyici olduğunu gösterir.
➤ İnsanlar yüksek gelirliyse ya da gelirine oranla düşük kredi istiyorsa, krediyi alma ihtimalleri çok daha yüksek.

🏠 2. Ev Sahipliği Durumu Önemlidir
	•	person_home_ownership_OWN, MORTGAGE, RENT, OTHER gibi ev sahipliği türleri yüksek öneme sahip.
➤ Sahip olunan ev (OWN: 0.3533) en olumlu sinyal olarak görülüyor.
➤ Ev sahibi olmak veya mortgage ile ev ödemek (MORTGAGE: 0.2040) pozitif etki yapıyor.
➤ Kiracı olmak (RENT: 0.1772) ya da diğer türler genelde daha düşük güven sağlıyor.

💰 3. Kredi Özellikleri de Güçlü Etkili
	•	loan_int_rate (0.34) ve loan_amnt (0.33) kredinin ne kadar riskli olduğu hakkında sinyal veriyor.
➤ Faiz oranı yüksekse veya kredi miktarı çok fazlaysa red ihtimali artıyor.

🧠 4. Eğitim Seviyesi Etkilidir
	•	Doktora (0.1551) ve yüksek lisans (0.1411) gibi yüksek eğitim seviyeleri öne çıkıyor.
➤ Ancak lise (0.1326) ve ön lisans (0.1299) da neredeyse aynı düzeyde.
➤ Buradan “düşük eğitimli” demek zor, çünkü eğitim çok belirleyici değil ama yine de önemli.

👨‍🔧 5. Çalışma Deneyimi ve Yaş Orta Seviye Etkili
	•	person_emp_exp: 0.1894
	•	person_age: 0.1906
➤ Yaş ve deneyim, kişinin finansal sorumluluğunu gösterdiği için mantıklı olarak etkili.

🧾 6. Kredi Amacı Önemli Sinyal
	•	Amaçlar arasında en riskli veya olumlu görülenler fark yaratıyor:
	•	HOMEIMPROVEMENT: 0.1759
	•	VENTURE: 0.1337 → Yeni iş kurma gibi riskli alanlar.
	•	DEBTCONSOLIDATION: 0.1500 → Borç birleştirme gibi amaçlar orta düzeyde riskli.

🧍‍♂️ 7. Cinsiyet ve Kredi Geçmişi Daha Az Etkili
	•	cb_person_cred_hist_length: 0.1412
	•	person_gender_male: 0.0853
➤ Kredi geçmişi hâlâ önemli ama üst sıralarda değil.
➤ Cinsiyetin etkisi oldukça düşük ve model açısından marjinal.

In [21]:
y_pred= model(x.to(device))
asdf =  np.where(y_pred.cpu().detach().numpy() > 0.5, 1, 0)
ydegerleri= y.cpu().detach().numpy()

In [22]:
asdf = pd.DataFrame(asdf)
ydegerleri = pd.DataFrame(ydegerleri)

In [23]:
karsilastirma = pd.concat([asdf,ydegerleri],axis=1)
karsilastirma.columns = ["gercek","tahmin"]

In [24]:
karsilastirma

,gercek,tahmin
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
495,1,1.0
496,0,0.0
497,0,0.0
498,0,0.0
